# Visualizing loss-landscapes

In [46]:
import numpy as np
import torch
import os
import matplotlib.pyplot as plt
import loss_landscapes
import loss_landscapes.metrics
import copy

In [47]:
#Initialize new model
class NeuralNetwork(torch.nn.Module):
    def __init__(self):
        """_summary_
        """
        super().__init__()
        self.linear_stack = torch.nn.Sequential(
            torch.nn.Linear(2, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 1)
        )

    def forward(self, x):
        """_summary_

        Args:
            x (_type_): _description_
            t (_type_): _description_

        Returns:
            _type_: _description_
        """
        return self.linear_stack(torch.Tensor(x))

In [48]:
#Load the model
loaded_model = torch.load("/Users/harry/1d-heat-pinn/src/model.pt")
loaded_model

NeuralNetwork(
  (linear_stack): Sequential(
    (0): Linear(in_features=2, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [49]:
model = NeuralNetwork()
model_initial = copy.deepcopy(model)

In [50]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

In [51]:
# setup time-space geometry
num_points = 51
x_start = 0
x_stop = 1
num_timesteps = 1000
t_start = 0
t_stop = 1 
x = np.linspace(start=x_start, stop=x_stop, num=num_points)
t = np.linspace(start=t_start, stop=t_stop, num=num_timesteps)

In [52]:
U_load = np.load("./1d-heat.npy")

features = np.ones((U_load.shape[0]*U_load.shape[1],2))
features[:,1] = np.tile(x,U_load.shape[0])
features[:,0] = np.repeat(t,U_load.shape[1])

labels = np.ones((U_load.shape[0]*U_load.shape[1],1))
labels = U_load.ravel()

labels = labels.astype(np.float32)
features = features.astype(np.float32)

In [53]:
# data that the evaluator will use when evaluating loss
features = torch.from_numpy(features).float()
labels = torch.from_numpy(labels).float()
metric = loss_landscapes.metrics.Loss(criterion, features, labels)

# compute loss data
STEPS = 40
loss_data = loss_landscapes.linear_interpolation(model_initial, loaded_model, metric, STEPS, deepcopy_model=True)

RuntimeError: expected scalar type Long but found Float